# Getting Started with Kubernetes / Microk8s

```
sudo snap install microk8s
sudo microk8s start
sudo microk8s enable dns storage gpu dashboard host-access ingress
sudo microk8s status
```

```
./run build
docker tag ocr2021 localhost:32000/ocr2021
docker push localhost:32000/ocr2021
```

In [1]:
cat > job.yml << \EOF
apiVersion: batch/v1
kind: Job
metadata:
  name: job$shard
spec:
  template:
    spec:
      containers:
      - name: job
        image: localhost:32000/ocr2021
        command:
        - /bin/bash
        - -c
        - |
          curl -sL http://storage.googleapis.com/ocropus4-g1000/Volume_$shard.tar |
          ocropus4 nlbin binarize - --output - |
          tar tvf - | sed 20q  #<-- for testing, we don't upload
      restartPolicy: Never
  backoffLimit: 4
EOF

In [25]:
export shard
for shard in {0000..0000}; do
    envsubst < job.yml
done

apiVersion: batch/v1
kind: Job
metadata:
  name: job0000
spec:
  template:
    spec:
      containers:
      - name: job
        image: localhost:32000/ocr2021
        command:
        - /bin/bash
        - -c
        - |
          curl -sL http://storage.googleapis.com/ocropus4-g1000/Volume_0000.tar |
          ocropus4 nlbin binarize - --output - |
          # gsutil cp - gs://ocropus4-temp
          tar tvf - | sed 20q
      restartPolicy: Never
  backoffLimit: 4


In [22]:
microk8s kubectl delete jobs --all
microk8s kubectl delete pods --all

No resources found
No resources found


In [23]:
export shard
for shard in {0000..0003}; do
    envsubst < job.yml | microk8s kubectl apply -f -
done

job.batch/job0000 created
job.batch/job0001 created
job.batch/job0002 created
job.batch/job0003 created


In [24]:
microk8s kubectl get pods

NAME            READY   STATUS              RESTARTS   AGE
job0000-776cr   0/1     ContainerCreating   0          7s
job0001-zfkvg   0/1     ContainerCreating   0          7s
job0002-vsjjq   0/1     ContainerCreating   0          6s
job0003-xvkxr   0/1     ContainerCreating   0          6s


In [26]:
job=$(microk8s kubectl get pods | awk '/^job.*Error|Running|Completed/{print $1}' | sed 1q)
echo $job
microk8s kubectl logs $job

job0000-776cr
-r--r--r-- bigdata/bigdata 171060 2021-08-25 22:22 Volume_0000/Image_0000.bin.png
-r--r--r-- bigdata/bigdata 692131 2021-08-25 22:22 Volume_0000/Image_0000.jpg
-r--r--r-- bigdata/bigdata 2484978 2021-08-25 22:22 Volume_0000/Image_0000.nrm.jpg
-r--r--r-- bigdata/bigdata   19879 2021-08-25 22:22 Volume_0000/Image_0001.bin.png
-r--r--r-- bigdata/bigdata  376956 2021-08-25 22:22 Volume_0000/Image_0001.jpg
-r--r--r-- bigdata/bigdata  561255 2021-08-25 22:22 Volume_0000/Image_0001.nrm.jpg
-r--r--r-- bigdata/bigdata  151595 2021-08-25 22:22 Volume_0000/Image_0002.bin.png
-r--r--r-- bigdata/bigdata  260388 2021-08-25 22:22 Volume_0000/Image_0002.jpg
-r--r--r-- bigdata/bigdata 1686126 2021-08-25 22:22 Volume_0000/Image_0002.nrm.jpg
-r--r--r-- bigdata/bigdata   98153 2021-08-25 22:22 Volume_0000/Image_0003.bin.png
-r--r--r-- bigdata/bigdata  172837 2021-08-25 22:22 Volume_0000/Image_0003.jpg
-r--r--r-- bigdata/bigdata 1242063 2021-08-25 22:22 Volume_0000/Image_0003.nrm.jpg
-r--r--r